# Формула расчёта смещения центров масс для ящика

### Исходные данные

In [82]:
# Характеристика 4-х осной ж/д платформы
L = 13400 # Длина пола
2870 # Ширина пола
Qv = 21 # Масса тары
wugr = 1310 # Высота пола от УГР
height_ct_vagon = 800 # Высота центра тяжести(ЦТ) от УГР
9720 # База платформы

# delta = [12168, 0, 3651, 7752] # расстояние между грузами (нет в записке)
delta = [690, 3055, 6930, 10915]

# Вес - в тоннах.
# Длина, ширина, высота - в мм.
cargo = [ # грузы
    {'length': 1080, 'width': 1580, 'height': 390, 'weight': 395 / 1000},  
    {'length': 3650, 'width': 3320, 'height': 1500, 'weight': 6670 / 1000},
    {'length': 4100, 'width': 1720, 'height': 1150, 'weight': 1865 / 1000},
    {'length': 3870, 'width': 2890, 'height': 1020, 'weight': 4085 / 1000},
]
weightSum = sum(map(lambda x: x['weight'], cargo))

### Формулы

In [83]:
for index, item in enumerate(cargo):
    item['ct'] = delta[index]

# Продольное смещение грузов в вагоне - в мм.
Lc = 0.5*L - (sum(map(lambda x: x['weight'] * x['ct'], cargo)) / weightSum)

# Продольное смещение грузов с вагоном - в мм.
Lcv = 0.5*L - (sum(map(lambda x: x['weight'] * x['ct'], cargo)) + Qv * L/2) / (weightSum + Qv)

# Общая высота ЦТ - в мм.
H = sum(map(lambda x: x['weight'] * (x['height']/2 + wugr), cargo)) / weightSum

print(Lc, Lcv, H)

694.4967345370733 265.7320299867706 1942.750672301191


In [71]:
cargo

[{'length': 1080, 'width': 1580, 'height': 390, 'weight': 0.395, 'ct': 1230.0},
 {'length': 3650, 'width': 3320, 'height': 1500, 'weight': 6.67, 'ct': 6650.0},
 {'length': 4100,
  'width': 1720,
  'height': 1150,
  'weight': 1.865,
  'ct': 17455.0},
 {'length': 3870,
  'width': 2890,
  'height': 1020,
  'weight': 4.085,
  'ct': 32355.0}]

### Доп формулы

In [8]:
# Общая высота ЦТ
height_center_gravity_in_car = (sum(map(lambda x: x['weight'] * (x['height']/2 + wugr), cargo)) + Qv * height_ct_vagon)/ (weightSum + Qv)
height_center_gravity_in_car

1237.2453329413495

In [10]:
# Расчет наветренной поверхности.
# Значение 7 - константа
sum([item['length'] / 1000 * item['height'] / 1000 for item in cargo]) + 7

21.5586

### Оптимизация

In [75]:
import random


# грузы
cargo = [
    {'length': 1080, 'width': 1580, 'height': 390, 'weight': 395 / 1000, 'delta': None},  
    {'length': 3650, 'width': 3320, 'height': 1500, 'weight': 6670 / 1000, 'delta': None},
    {'length': 4100, 'width': 1720, 'height': 1150, 'weight': 1865 / 1000, 'delta': None},
    {'length': 3870, 'width': 2890, 'height': 1020, 'weight': 4085 / 1000, 'delta': None},
]
weightSum = sum(map(lambda x: x['weight'], cargo))

# Свободное место на платформе, если расположить все грузы.
free_L = L - sum([item['length'] for item in cargo])
print(f'{free_L = }')
# Даём пустое пространство на один груз, где можем его двигать.
free_L_for_one_cargo = free_L / len(cargo)
print(f'{free_L_for_one_cargo = }')

if free_L < 0:
    print('Груз не влазит!')

free_L = 700
free_L_for_one_cargo = 175.0


In [84]:
class Optimize:
    """ Модель для оптимизации продольного смещения. """

    def __init__(self, cargo: list, step=1, border_epsilon_step=1, count_early_stop=100):
        """ step - шаг влево/вправо для оптимизации delta. """
        
        self.best_cargo = None
        self.score = float('inf')

        self._step = step
        self._border_epsilon_step = border_epsilon_step  # Рандомное значение добавляется к step.
        self._count_early_stop = count_early_stop
        
        self.result = self._optimize(cargo)


    def _init_delta(self, cargo: list, start_idx=0):
        """ Инициализация значений delta. Внутри создаёт новый cargo. """
    
        # copy_cargo = copy.deepcopy(cargo)
        copy_cargo = [{k:v for k, v in dct.items()} for dct in cargo]

        # Меняю очерёдность грузов на платформе.
        copy_cargo = copy_cargo[start_idx:] + copy_cargo[:start_idx]
        
        if len(copy_cargo) == 1:
            copy_cargo[0]['delta'] = L / 2 - copy_cargo[0]['length'] / 2
    
        else:
    
            prev_border = 0
            for idx, item in enumerate(copy_cargo):
    
                copy_cargo[idx]['delta'] = random.randint(prev_border, prev_border + free_L_for_one_cargo - 1)
    
                # Обновляем левую границу.
                prev_border = prev_border + item['length'] + free_L_for_one_cargo
    
        return copy_cargo
    
    
    def _get_score(self, cargo: list):
        """ Получение метрик при изменённом delta у одного груза. """

        # Продольное смещение грузов в вагоне - в мм.
        Lc = 0.5*L - (sum(map(lambda x: x['weight'] * x['delta'], cargo)) / weightSum)
    
        return Lc
    
    def _check_best_cargo(self, cargo: list):
        """ Проверка метрик у обновлённого delta у cargo. """

        score = self._get_score(cargo)
        if abs(score) < abs(self.score):
            self.best_cargo = cargo
            self.score = score
            return True

        return False
    
    def _optimize(self, cargo: list):
        """ Процесс оптимизации delta. """

        # Меняю очерёдность расположение каждого груза.
        for i in range(0, len(cargo)):
            print(f'Началась оптимизация, где первый груз с индексом {i}')
            copy_cargo = self._init_delta(cargo, i)

            # Эпохи:
            counter = 0
            while counter < self._count_early_stop:

                # Смещаю каждый груз влево и вправо на step, затем смотрю метрики.
                for k in range(len(copy_cargo)):

                    left_step_cargo = [{k:v for k, v in dct.items()} for dct in copy_cargo]
                    new_step = self._step + random.randint(0, self._border_epsilon_step)
                    left_step_cargo[k]['delta'] -= new_step
                    
                    right_step_cargo = [{k:v for k, v in dct.items()} for dct in copy_cargo]
                    new_step = self._step + random.randint(0, self._border_epsilon_step)
                    right_step_cargo[k]['delta'] += new_step

                    # Проверки на не наложение одного груза на другой.
                    # Нулевый элемент не выходит за пределы платформы слева.
                    if k == 0:
                        if left_step_cargo[k]['delta'] >= 0:
                            # проверка
                            if self._check_best_cargo(left_step_cargo):
                                counter = 0
                                copy_cargo = self.best_cargo
                                
                    # Последний элемент не выходит за пределы платформы справа.
                    elif k == len(copy_cargo) - 1:
                        if right_step_cargo[k]['delta'] + right_step_cargo[k]['length'] < L:
                            # проверка
                            if self._check_best_cargo(right_step_cargo):
                                counter = 0
                                copy_cargo = self.best_cargo
                            
                    else:
                        # Проверка на не наложение правого груза на левый. 
                        if left_step_cargo[k]['delta'] > left_step_cargo[k-1]['delta'] + left_step_cargo[k-1]['length']:
                            if self._check_best_cargo(left_step_cargo):
                                counter = 0
                                copy_cargo = self.best_cargo

                        # Проверка на не наложение левого груза на превый. 
                        if right_step_cargo[k]['delta'] + right_step_cargo[k]['length'] < right_step_cargo[k+1]['delta']:
                            if self._check_best_cargo(right_step_cargo):
                                counter = 0
                                copy_cargo = self.best_cargo

                # После каждой эпохи увеличиваю.
                counter += 1
            print(self.score)        

        return self.best_cargo, self.score


tmp = Optimize(cargo, step=1, border_epsilon_step=5, count_early_stop=1000)
print(tmp.score)

Началась оптимизация, где первый груз с индексом 0
2020.0099884748379
Началась оптимизация, где первый груз с индексом 1
2020.0099884748379
Началась оптимизация, где первый груз с индексом 2
-0.003841721090793726
Началась оптимизация, где первый груз с индексом 3
-0.003841721090793726
-0.003841721090793726


In [77]:
tmp.best_cargo

[{'length': 4100,
  'width': 1720,
  'height': 1150,
  'weight': 1.865,
  'delta': 106,
  'ct': 106},
 {'length': 3870,
  'width': 2890,
  'height': 1020,
  'weight': 4.085,
  'delta': 4557,
  'ct': 4557},
 {'length': 1080,
  'width': 1580,
  'height': 390,
  'weight': 0.395,
  'delta': 8510,
  'ct': 8510},
 {'length': 3650,
  'width': 3320,
  'height': 1500,
  'weight': 6.67,
  'delta': 9749,
  'ct': 9749}]